In [4]:
from __future__ import unicode_literals, print_function, division

import os
from io import open
import sys
import math
import random
import argparse
import operator
import pdb

import torch
import torch.autograd as autograd
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence


from collections import defaultdict
from collections import Counter

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


# Kyle's attempt
import faker
from faker import Faker
import pandas as pd
import numpy as np
import re
from string import punctuation
import glob
import unicodedata
import string
import random

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

fake = Faker()

In [5]:
class LSTMClassifier(nn.Module):

    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_size):
        super(LSTMClassifier, self).__init__()

        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=1)

        self.hidden2out = nn.Linear(hidden_dim, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

        self.dropout_layer = nn.Dropout(p=0.2)


    def init_hidden(self, batch_size):
        return(autograd.Variable(torch.randn(1, batch_size, self.hidden_dim)),
                    autograd.Variable(torch.randn(1, batch_size, self.hidden_dim)))


    def forward(self, batch, lengths):

        self.hidden = self.init_hidden(batch.size(-1))

        embeds = self.embedding(batch)
        packed_input = pack_padded_sequence(embeds, lengths)
        outputs, (ht, ct) = self.lstm(packed_input, self.hidden)
        # ht is the last hidden state of the sequences
        # ht = (1 x batch_size x hidden_dim)
        # ht[-1] = (batch_size x hidden_dim)
        output = self.dropout_layer(ht[-1])
        output = self.hidden2out(output)
        output = self.softmax(output)

        return output

In [6]:
        
class PaddedTensorDataset(Dataset):
#     """Dataset wrapping data, target and length tensors.

#     Each sample will be retrieved by indexing both tensors along the first
#     dimension.

#     Arguments:
#         data_tensor (Tensor): contains sample data.
#         target_tensor (Tensor): contains sample targets (labels).
#         length (Tensor): contains sample lengths.
#         raw_data (Any): The data that has been transformed into tensor, useful for debugging
#     """

    def __init__(self, data_tensor, target_tensor, length_tensor, raw_data):
        assert data_tensor.size(0) == target_tensor.size(0) == length_tensor.size(0)
        self.data_tensor = data_tensor
        self.target_tensor = target_tensor
        self.length_tensor = length_tensor
        self.raw_data = raw_data

    def __getitem__(self, index):
        return self.data_tensor[index], self.target_tensor[index], self.length_tensor[index], self.raw_data[index]

    def __len__(self):
        return self.data_tensor.size(0)

In [7]:
class DF_To_Tensors():
    def __init__(self):
        self.tag2id = defaultdict(int,
                        {'city': 0,
                         'first_name': 1,
                         'geo': 2,
                         'percent': 3,
                         'year': 4,
                         'date': 5,
                         'ssn': 6,
                         'language_name': 7,
                         'country': 8,
                         'phone_number': 9,
                         'month': 10,
                         'zipcode': 11,
                         'iso8601': 12,
                         'paragraph': 13,
                         'pyfloat': 14,
                         'email': 15,
                         'prefix': 16,
                         'pystr': 17,
                         'isbn': 18,
                         'boolean': 19,
                        'country_code': 20,
                        'continent':21})
        self.n_categories = len(self.tag2id)
        self.token_set={'a','b','c','d','e',
                        'f','g','h','i','j','k','l',
                        'm','n','o','p','q','r','s',
                        't','u','v','w','x','y','z',
                        'A','B','C','D','E','F','G',
                        'H','I','J','K','L','M','N',
                        'O','P','Q','R','S','T','U',
                        'V','W','X','Y','Z','1','2',
                        '3','4','5','6','7','8','9','0',
                        "'",',','.',';','*','!','@',
                        '#','$','%','^','&','(',')',
                        '_','=','-',':','+','/',"\\", '*'}
        self.token2id = defaultdict(int,
            {'PAD': 0,
             'UNK': 1,
             'a':2,
             'b':3,
             'c': 4,
             'd': 5,
             'e': 6,
             'f': 7,
             'g':8,
             'h': 9,
             'i': 10,
             'j':11,
             'k':12,
             'l':13,
             'm':14,
             'n':15,
             'o':16,
             'p':17,
             'q':18,
             'r':19,
             's':20,
             't':21,
             'u':22,
             'v':23,
             'w':24,
             'x':25,
             'y':26,
             'z':27,
             'A':28,
             'B':29,
             'C':30,
             'D':31,
             'E':32,
             'F':33,
             'G':34,
             'H':35,
             'I':36,
             'J':37,
             'K':38,
             'L':39,
             'N':40,
             'O':41,
             'P':42,
             'Q':43,
             'R':44,
             'S':45,
             'T':46,
             'U':47,
             'V':48,
             'W':49,
             'X':50,
             'Y':51,
             'Z':52,
             '1':53,
             '2':54,
             '3':55,
             '4':56,
             '5':57,
             '6':58,
             '7':59,
             '8':60,
             '9':61,
             '0':62,
             "'":63,
             ',':64,
             '.':65,
             ';':66,
             '*':67,
             '!':68,
             '@':68,
             '#':70,
             '$':71,
             '%':72,
             '^':73,
             '&':74,
             '(':75,
             ')':76,
             '_':77,
             '=':78,
             '-':79,
             ':':80,
             '+':81,
             '/':82,
             '\\':83,
             '*': 84})
    
    def vectorized_string(self, string):
            return [self.token2id[token] if token in self.token2id else self.token2id['UNK'] for token in str(string)]
        
    def vectorized_array(self, array):
        vecorized_array=[]
        for stringValue in array:
            vecorized_array.append(self.vectorized_string(stringValue))
        return vecorized_array
    
    def pad_sequences(self, vectorized_seqs, seq_lengths):
        # create a zero matrix
        seq_tensor = torch.zeros((len(vectorized_seqs), seq_lengths.max())).long()

        # fill the index
        for idx, (seq, seqlen) in enumerate(zip(vectorized_seqs, seq_lengths)):
            seq_tensor[idx, :seqlen] = torch.LongTensor(seq)
        return seq_tensor


    def create_dataset(self, data, batch_size=1):
        vectorized_seqs = self.vectorized_array(data)
        seq_lengths = torch.LongTensor([len(s) for s in vectorized_seqs])
        seq_tensor = self.pad_sequences(vectorized_seqs, seq_lengths)
        target_tensor = torch.LongTensor([self.tag2id[y] for  y in data])
        raw_data = [x for x in data]
        
        return DataLoader(PaddedTensorDataset(seq_tensor, target_tensor, seq_lengths, raw_data), batch_size=batch_size)

    def sort_batch(self,batch, targets, lengths):
        seq_lengths, perm_idx = lengths.sort(0, descending=True)
        seq_tensor = batch[perm_idx]
        target_tensor = targets[perm_idx]

        return seq_tensor.transpose(0, 1), target_tensor, seq_lengths


    def evaluate_test_set(self, model, test):
        y_pred = list()
        all_predictionsforValue=[]

        for batch, targets, lengths, raw_data in self.create_dataset(test, batch_size=1):
            batch, targets, lengths = self.sort_batch(batch, targets, lengths)
            pred = model(torch.autograd.Variable(batch), lengths.cpu().numpy())
            pred_idx = torch.max(pred, 1)[1]
            print('pred_idx', pred_idx)
            def get_key(val):
                for key, value in self.tag2id.items():
                     if val == value:
                            print(key)
                            return {'key':key, 'tensor':pred, 'pred_idx':pred_idx}
#                             all_predictionsforValue.append({'key':key, 'tensor':pred, 'pred_idx':pred_idx})

            all_predictionsforValue.append(get_key(pred_idx[0]))
        return all_predictionsforValue
        
    def read_in_csv(self,path):
        self.df = pd.read_csv(path)
#         print(self.df.head())
    
    def get_arrayOfValues_df(self):
        import time
        column_value_object={}

        for column in self.df.columns:
            guesses=[]
            column_value_object[column]=[]
            for _ in range(1,3):
                random_values = str(np.random.choice(self.df[column]))
                random_col = column
                column_value_object[column].append(str(random_values)) 

        return column_value_object
    
    def predictions(self, model, path_to_csv):
        self.read_in_csv(path=path_to_csv)
        column_value_object = self.get_arrayOfValues_df()
        for column in column_value_object:
            print(column_value_object[column])
            print(column)
            print(self.evaluate_test_set(model, column_value_object[column]))
#             return {
#                 'column': column,
#                 'values': column_value_object[column],
#                 'predictions': self.evaluate_test_set(column_value_object[column])
#             }
        


In [10]:
# create our class and load the saved model
dft_tensor=DF_To_Tensors()
model = torch.load('./models/LSTM_RNN_classifier_model_5.pth')
model.eval()

LSTMClassifier(
  (embedding): Embedding(84, 128)
  (lstm): LSTM(128, 32)
  (hidden2out): Linear(in_features=32, out_features=22, bias=True)
  (softmax): LogSoftmax(dim=1)
  (dropout_layer): Dropout(p=0.2, inplace=False)
)

In [ ]:
dft_tensor.predictions(model=model, path_to_csv='datasets/data/africa_test.csv')

In [ ]:
dft_tensor.predictions(model=model, path_to_csv='datasets/data/four_col_test.csv')

In [15]:
# Just test a random array by itself.
dft_tensor.evaluate_test_set(model=model,test=['01/02/2020', '09', 'USA', 'WOWOOWOWOWOWOWOWO', '1999', '1200', '402', '.3934', 'iiii', '12', 'USA',"ETH",'South America'])

In [41]:

# for later use
def randomChoice(self, values):
    return values[random.randint(0, len(values) - 1)]

def getRandomSet(self):
    category = self.randomChoice(self.all_categories)
#         print(category)
    line = self.randomChoice(list(self.category_values[category]['obj']))
#         print('line', line)
    return (line, category)